In [5]:
'''
import packages we need
'''
import pandas as pd
import csv
import numpy as np

In [39]:
df = pd.read_csv("weight_all_gz.csv")#, encoding='latin-1') 

/opt/miniconda3/envs/d2l/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [40]:
'''
show us the first five lines of assembled POIs
'''
df.head()

,步行最短距离,步行最短时间,公共服务设施名称,ID,距离分类,权重,纠正后权重
0,155,124,鸿力游泳健身俱乐部,1.0,1.00,0.08,0.08
1,298,238,源力汇健身拳击,1.0,1.00,0.08,0.08
2,448,358,Salute健身工作室,1.0,0.90,0.08,0.072
3,651,521,维拉游泳健身俱乐部,1.0,0.90,0.08,0.072
4,836,669,CGV影城(北京路店),1.0,0.55,0.08,0.044


In [41]:
dff = pd.read_csv("street_2.0.csv",header = None)

In [43]:
'''
assemble all pois of one address and calculate its grades,save it in dictionary 'data'
'''
data = {}
for i in range(1,172):#range(1,230)
    df_1 = df[(df['ID'] == i)]
    df_1c = df_1['纠正后权重']
    sum1 = 0
    for j in list(df_1c):
        try:
            i_f = float(j)
        except ValueError:
            i_f = float(j[:-2])
        sum1 += float(i_f)
    #print(sum1)
    data[i] = sum1

In [45]:
max(data.values())

1790.2318000000037

In [46]:
min(data.values())

0.324

In [47]:
'''
show data in DataFrame format
'''
data_df = pd.DataFrame(data,index=[0]).T
data_df

,0
1,950.4573
2,75.8810
3,574.4654
4,63.7165
5,387.7669
...,...
167,5.9460
168,251.5325
169,853.9726
170,208.4239


In [48]:
data_df[0]

1      950.4573
2       75.8810
3      574.4654
4       63.7165
5      387.7669
         ...   
167      5.9460
168    251.5325
169    853.9726
170    208.4239
171     42.9695
Name: 0, Length: 171, dtype: float64

In [49]:
'''
here are some basic information of scores after distance-based decay function 
'''
data_df.describe()

,0
count,171.000000
mean,417.153548
std,345.528837
min,0.324000
25%,141.133350
50%,346.061000
75%,573.417350
max,1790.231800


In [50]:
'''
add a new column
'''
dff['3']=''

In [51]:
'''
block-characteristic-based decay function
'''
def fix(x,y):
    if x < 50:
        wei = 0.95
    elif x < 80 and x >= 50:
        wei = 0.96
    elif x < 120 and x >= 80:
        wei = 0.97
    elif x < 150 and x >= 120:
        wei = 0.98
    elif x < 200 and x >= 150:
        wei = 0.99
    else:
        wei = 1

    if y < 100:
        ght = 1
    elif x < 120 and x >= 100:
        ght = 0.99
    elif x < 150 and x >= 120:
        ght = 0.98
    elif x < 180 and x >= 150:
        ght = 0.97
    elif x < 200 and x >= 180:
        ght = 0.96
    else:
        ght = 0.95
    return(wei*ght)

In [52]:
'''
calculate scores after considering block characteristics
'''
for i in data:
    dff.iloc[[i-1],[3]]=data[i]
    line = dff.iat[i-1, 2]
    line = line.replace('[', '')
    line = line.replace(']', '')
    line = line.split(',')
    #print(line)
    x = float(line[0])
    y = float(line[1])
    dff.iloc[[i-1],[3]]=data[i]*fix(x,y)

In [53]:
'''
show us the first five lines of assembled POIs
'''
dff.head()

,0,1,2,3
0,"113.275128,23.117633",珠光街道,"[182.1003530684274, 79.21584647302905]",940.952727
1,"113.177516,23.426931",秀全街道,"[67.60406911970202, 136.25749999999996]",69.203472
2,"113.322082,23.166966",兴华街道,"[5.3026609586811615, 375.12302597402606]",518.455024
3,"113.532512,23.593680",城郊街道,"[8.418909233086419, 337.739780361757]",57.504141
4,"113.599693,23.545329",江埔街道,"[5.566734325055002, 213.8208961038963]",349.959627


In [65]:
'''
save file
'''
dff.to_csv("score.csv",index=False,sep=',')

In [64]:
max(dff['3'])

1772.3294820000035

In [57]:
'''
extract final scores 
'''
dff_1=pd.DataFrame(dff['3'],dtype=np.float)

In [58]:
'''
show us basic information of the final scores
'''
dff_1.describe()

,3
count,171.000000
mean,389.361230
std,331.382889
min,0.292410
25%,129.171711
50%,312.320052
75%,524.141613
max,1772.329482


In [59]:
'''
only keep three digital numbers
'''
dff_1.round(3).describe()

,3
count,171.000000
mean,389.361228
std,331.382893
min,0.292000
25%,129.172000
50%,312.320000
75%,524.141500
max,1772.329000


In [60]:
#中位数
np.median(dff_1.round(3))

312.32

In [61]:
from scipy import stats
'''
use stats package to calculate mode
'''
stats.mode(dff_1.round(0))[0][0]

array([58.])

In [68]:
'''
a new score.csv, after using Excel to standardize scores into [0,100]
'''
score = pd.read_csv("score.csv")

In [69]:
score.head()

,0,1,2,3
0,"113.275128,23.117633",珠光街道,"[182.1003530684274, 79.21584647302905]",940.952727
1,"113.177516,23.426931",秀全街道,"[67.60406911970202, 136.25749999999996]",69.203472
2,"113.322082,23.166966",兴华街道,"[5.3026609586811615, 375.12302597402606]",518.455024
3,"113.532512,23.593680",城郊街道,"[8.418909233086419, 337.739780361757]",57.504141
4,"113.599693,23.545329",江埔街道,"[5.566734325055002, 213.8208961038963]",349.959627


In [54]:
'''
show us basic statistic information about standardized final scores
'''
score['4'].round(2).describe()
Out[72]:
count    171.000000
mean      22.003743
std       18.733260
min        0.000000
25%        7.295000
50%       17.690000
75%       29.615000
max      100.000000
Name: 4, dtype: float64
In [74]:
np.median(score['4'].round(2))
Out[74]:
17.69
In [75]:
stats.mode(score['4'].round(0))[0][0]
Out[75]:
29.0
In [79]:
'''
calculate the numbers of communities that fall into different score scope
'''
#：[(10, 20] < (20, 30] < (30, 50] < (50, 70] < (70, 80] < (80, 90]]
scores_bins = [-0.01,24.99,49.99,69.99,89.99,100.01]
a,b = pd.cut(x=score["4"],bins=scores_bins,right=True,retbins=True)
In [80]:
a
Out[80]:
0      (49.99, 69.99]
1      (-0.01, 24.99]
2      (24.99, 49.99]
3      (-0.01, 24.99]
4      (-0.01, 24.99]
            ...      
166    (-0.01, 24.99]
167    (-0.01, 24.99]
168    (24.99, 49.99]
169    (-0.01, 24.99]
170    (-0.01, 24.99]
Name: 4, Length: 171, dtype: category
Categories (5, interval[float64]): [(-0.01, 24.99] < (24.99, 49.99] < (49.99, 69.99] < (69.99, 89.99] < (89.99, 100.01]]
In [84]:
a.value_counts()
Out[84]:
(-0.01, 24.99]     106
(24.99, 49.99]      50
(49.99, 69.99]      10
(69.99, 89.99]       4
(89.99, 100.01]      1
Name: 4, dtype: int64
In [ ]:
 

SyntaxError: invalid syntax (<ipython-input-54-1ed03acce396>, line 5)

In [4]:
#  遍历读取文件夹中的每一个文件
fileStart = 1
fileEnd = len(file_list)
for i in range(fileStart,fileEnd):
    df_path = Folder_Path + '/' + file_list[i]
    if df_path.endswith('.csv'):
        df = pd.read_csv(df_path, 
                   encoding = "utf8")
        df.to_csv (SaveFile_Path + '/' + SaveFile_Name,
                   encoding = "utf8",
                   index = False,
                   header = True,#如果对每个文件的标题另外有需求，需要对header变量进行指定
                   mode = 'a')
        print('共合并%i个CSV文件'% len(file_list))

共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
共合并28个CSV文件
